# Day 15: Beacon Exclusion Zone

You feel the ground rumble again as the distress signal leads you to a large network of subterranean tunnels. You don't have time to search them all, but you don't need to: your pack contains a set of deployable sensors that you imagine were originally built to locate lost Elves.

The sensors aren't very powerful, but that's okay; your handheld device indicates that you're close enough to the source of the distress signal to use them. You pull the emergency sensor system out of your pack, hit the big button on top, and the sensors zoom off down the tunnels.

Once a sensor finds a spot it thinks will give it a good reading, it attaches itself to a hard surface and begins monitoring for the nearest signal source beacon. Sensors and beacons always exist at integer coordinates. Each sensor knows its own position and can determine the position of a beacon precisely; however, sensors can only lock on to the one beacon closest to the sensor as measured by the Manhattan distance. (There is never a tie where two beacons are the same distance to a sensor.)

It doesn't take long for the sensors to report back their positions and closest beacons (your puzzle input). For example:

```
Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3
```

In [64]:
from dataclasses import dataclass
from typing import List, Tuple, Iterator
import parse

@dataclass
class Position:
    x: int
    y: int
    
        
def read_sensor_file(file_name: str) -> Iterator[Position]:
    p = parse.compile("Sensor at x={:d}, y={:d}: closest beacon is at x={:d}, y={:d}")
    with open(file_name) as f:
        for line in f.read().splitlines():
            r = p.parse(line)
            yield Position(*r[:2]), Position(*r[2:])

In [65]:
data = list(read_sensor_file("example"))

From [wikipedia](https://en.wikipedia.org/wiki/Taxicab_geometry)

In $\mathbb{R}^2$, the taxicab distance between two points $(x_1, y_1)$ and $(x_2, y_2)$ is $\left|x_1 - x_2\right| + \left|y_1 - y_2\right|$. That is, it is the sum of the absolute values of the differences in both coordinates.

In [66]:
def manhattan(p1: Position, p2: Position) -> int:
    return abs(p1.x - p2.x) + abs(p1.y - p2.y)

In [67]:
# each sensor and its radius of detection
PosDistance = Tuple[Position, Position, int]

distances_from_sensor : List[PosDistance]
distances_from_sensor = [(p[0], p[1], manhattan(*p)) for p in data]
distances_from_sensor

[(Position(x=2, y=18), Position(x=-2, y=15), 7),
 (Position(x=9, y=16), Position(x=10, y=16), 1),
 (Position(x=13, y=2), Position(x=15, y=3), 3),
 (Position(x=12, y=14), Position(x=10, y=16), 4),
 (Position(x=10, y=20), Position(x=10, y=16), 4),
 (Position(x=14, y=17), Position(x=10, y=16), 5),
 (Position(x=8, y=7), Position(x=2, y=10), 9),
 (Position(x=2, y=0), Position(x=2, y=10), 10),
 (Position(x=0, y=11), Position(x=2, y=10), 3),
 (Position(x=20, y=14), Position(x=25, y=17), 8),
 (Position(x=17, y=20), Position(x=21, y=22), 6),
 (Position(x=16, y=7), Position(x=15, y=3), 5),
 (Position(x=14, y=3), Position(x=15, y=3), 1),
 (Position(x=20, y=1), Position(x=15, y=3), 7)]

So, consider the sensor at 2,18; the closest beacon to it is at -2,15. For the sensor at 9,16, the closest beacon to it is at 10,16.

Drawing sensors as S and beacons as B, the above arrangement of sensors and beacons looks like this:

```
               1    1    2    2
     0    5    0    5    0    5
 0 ....S.......................
 1 ......................S.....
 2 ...............S............
 3 ................SB..........
 4 ............................
 5 ............................
 6 ............................
 7 ..........S.......S.........
 8 ............................
 9 ............................
10 ....B.......................
11 ..S.........................
12 ............................
13 ............................
14 ..............S.......S.....
15 B...........................
16 ...........SB...............
17 ................S..........B
18 ....S.......................
19 ............................
20 ............S......S........
21 ............................
22 .......................B....
```

This isn't necessarily a comprehensive map of all beacons in the area, though. Because each sensor only identifies its closest beacon, if a sensor detects a beacon, you know there are no other beacons that close or closer to that sensor. There could still be beacons that just happen to not be the closest beacon to any sensor. Consider the sensor at 8,7:

```
               1    1    2    2
     0    5    0    5    0    5
-2 ..........#.................
-1 .........###................
 0 ....S...#####...............
 1 .......#######........S.....
 2 ......#########S............
 3 .....###########SB..........
 4 ....#############...........
 5 ...###############..........
 6 ..#################.........
 7 .#########S#######S#........
 8 ..#################.........
 9 ...###############..........
10 ....B############...........
11 ..S..###########............
12 ......#########.............
13 .......#######..............
14 ........#####.S.......S.....
15 B........###................
16 ..........#SB...............
17 ................S..........B
18 ....S.......................
19 ............................
20 ............S......S........
21 ............................
22 .......................B....
```

This sensor's closest beacon is at 2,10, and so you know there are no beacons that close or closer (in any positions marked #).

None of the detected beacons seem to be producing the distress signal, so you'll need to work out where the distress beacon is by working out where it isn't. For now, keep things simple by counting the positions where a beacon cannot possibly be along just a single row.

So, suppose you have an arrangement of beacons and sensors like in the example above and, just in the row where y=10, you'd like to count the number of positions a beacon cannot possibly exist. The coverage from all sensors near that row looks like this:

```
                 1    1    2    2
       0    5    0    5    0    5
 9 ...#########################...
10 ..####B######################..
11 .###S#############.###########.
```

In this example, in the row where y=10, there are 26 positions where a beacon cannot be present.

Consult the report from the sensors you just deployed. In the row where y=2000000, how many positions cannot contain a beacon?


In [68]:
def is_inreach_of_a_sensor(sensors: List[PosDistance], pos: Position) -> bool:
    for sensor, beacon, radius in sensors:
        this_distance = manhattan(pos, sensor)
        if this_distance < radius:  # < and not <= because only one beacon is ever detected
            return True
        elif pos == beacon:
            return True
    return False

is_inreach_of_a_sensor(distances_from_sensor, Position(-3, 10))

False

In [56]:
# find the relevant sensors for a given row
row = 10
relevant_sensors : List[PosDistance]
relevant_sensors = []
for (pos, dist) in distances_from_sensor:
    if abs(pos.y - row) <= dist:
        relevant_sensors.append((pos, dist))

In [57]:
relevant_sensors

[(Position(x=12, y=14), 4),
 (Position(x=8, y=7), 9),
 (Position(x=2, y=0), 10),
 (Position(x=0, y=11), 3),
 (Position(x=20, y=14), 8),
 (Position(x=16, y=7), 5)]

In [58]:
# find the min and max x position in this row, that can be reached by a sensor
# (to make it easy we will consider a way too large interval)
x_min = min(p.x - d for (p, d) in relevant_sensors)
x_max = max(p.x + d for (p, d) in relevant_sensors)

x_min, x_max

(-8, 28)

In [59]:
# Now we just have to scan the row from x_min to x_max
for x in range(x_min, x_max + 1):
    print(x, end=", ")
    is_covered = is_inreach_of_a_sensor(relevant_sensors, Position(x, row))
    print(is_covered)

res = [is_inreach_of_a_sensor(relevant_sensors, Position(x, row)) for x in range(x_min, x_max + 1)]
sum(res)

-8, False
-7, False
-6, False
-5, False
-4, False
-3, False
-2, False
-1, True
0, True
1, True
2, False
3, True
4, True
5, True
6, True
7, True
8, True
9, True
10, True
11, True
12, True
13, True
14, False
15, True
16, True
17, True
18, True
19, True
20, True
21, True
22, True
23, True
24, False
25, False
26, False
27, False
28, False


23